In [55]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import pandas as pd
import itertools
import sys
import os
import math
sys.path.append('../..')
import scipy.optimize as opt
from joblib import Parallel, delayed
from functools import partial
from hyperopt import hp, tpe, fmin, Trials
from models.optim.sir_simple import SIR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# np.array([start days]), np.array([duration]), np.array([choice], number_of_days_simulated)
def calculate_opt(intervention_day, intervention_duration, intervention_choice, days):
    R0 = 2.2 
    T_inf = 2.9
    T_trans = T_inf/R0
    T_treat = 15

    N = 1e5
    I0 = 3.0

    assert(len(intervention_day) == len(intervention_duration))
    assert(len(intervention_duration) == len(intervention_choice))
    
    params = [T_trans, T_treat, N, intervention_day, intervention_duration, intervention_choice]

    # S, E, I, R_mild, R_severe, R_severe_home, R_fatal, C, D
    state_init_values = [(N - I0)/N, I0/N, 0]
    
    solver = SEIR(params, state_init_values)
    sol = solver.solve_ode(time_step=1, total_no_of_days=days)
    states_int_array = (sol.y*N).astype('int')
    
    
    S_coeficeint=0
    I_coeficeint=1
    R_coeficeint=0
    
    
    coeficeint=[S_coeficeint,I_coeficeint,R_coeficeint]
    
    for i in range(days):
        grad1 = np.dot(coeficeint,np.sum(states_int_array/N, axis=1))
    return(grad1)

In [23]:
fx = calculate_opt(intervention_day=np.array([100]), intervention_duration=np.array([50]), intervention_choice=np.array([0]), days=400)
print(fx)

14.99909


In [24]:
def check(start_array, duration_array, choice_array, total_resource):
    for i in range(1,len(start_array)):
        if(start_array[i] <= start_array[i-1] + duration_array[i-1]):
            return(0)
    resource_spent = np.dot(duration_array, choice_array)
    if(resource_spent != total_resource):
        return(0)
    else:
        return(1)
    

def grid_search(num_int, total_resource=30, day0=30):
    min_val = 100
    min_params = {}
    params = []
    min_duration = 10
    max_duration = total_resource*np.array([4,2,1])
   
    if(num_int == 1):
        start_val = [start for start in range(day0, day0+365, 10)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource, 5)]
        start_tuple = list(itertools.product(start_val))
        choice_tuple = list(itertools.product(choice_val))
        duration_tuple = list(itertools.product(duration_val))
        inputs = list(itertools.product(start_tuple, duration_tuple, choice_tuple))
        for inp in inputs:
            start_array = np.array(inp[0])
            duration_array = np.array(inp[1])
            choice_array = np.array(inp[2])
            if(check(start_array, duration_array, choice_array, total_resource)):
                params.append([start_array, choice_array, duration_array])
        
    if(num_int == 2):
        start_val = [start for start in range(day0, day0+365, 10)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource, 5)]
        start_tuple = list(itertools.product(start_val, start_val))
        choice_tuple = list(itertools.product(choice_val, choice_val))
        duration_tuple = list(itertools.product(duration_val, duration_val))
        inputs = list(itertools.product(start_tuple, duration_tuple, choice_tuple))
        for inp in inputs:
            start_array = np.array(inp[0])
            duration_array = np.array(inp[1])
            choice_array = np.array(inp[2])
            if(check(start_array, duration_array, choice_array, total_resource)):
                params.append([start_array, choice_array, duration_array])
            
    if(num_int == 3):
        start_val = [start for start in range(day0, day0+365, 40)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource, 10)]
        start_tuple = list(itertools.product(start_val, start_val, start_val))
        choice_tuple = list(itertools.product(choice_val, choice_val, choice_val))
        duration_tuple = list(itertools.product(duration_val, duration_val, duration_val))
        inputs = list(itertools.product(start_tuple, duration_tuple, choice_tuple))
        for inp in inputs:
            start_array = np.array(inp[0])
            duration_array = np.array(inp[1])
            choice_array = np.array(inp[2])
            if(check(start_array, duration_array, choice_array, total_resource)):
                params.append([start_array, choice_array, duration_array])
        
                            
    print(len(params))

    value_array = Parallel(n_jobs=40)(delayed(calculate_opt)(intervention_day=par[0], intervention_duration=par[2],\
                                                             intervention_choice=par[1], days = 400) for par in params)
    value_array = np.array(value_array)
    min_val = np.min(value_array)
    i = np.argmin(value_array)
    min_params['start_array'] = params[i][0]
    min_params['duration_array'] = params[i][2]
    min_params['choice_array'] = params[i][1]
    
    return(min_val, min_params)    

In [25]:
#num_int = 1
grid_search(num_int=1)

74


(14.99759,
 {'start_array': array([30]),
  'duration_array': array([30]),
  'choice_array': array([1])})

In [26]:
# num_int = 2
grid_search(num_int=2)

36667


(14.997160000000001,
 {'start_array': array([30, 50]),
  'duration_array': array([15, 15]),
  'choice_array': array([1, 1])})

In [27]:
#num_int = 3
grid_search(num_int=3)

23048


(14.997770000000001,
 {'start_array': array([ 30,  70, 110]),
  'duration_array': array([20, 10, 10]),
  'choice_array': array([1. , 0.5, 0.5])})

In [56]:
def check(start_array, duration_array, choice_array, total_resource):
    for i in range(1,len(start_array)):
        if(start_array[i] <= start_array[i-1] + duration_array[i-1]):
            return(0)
    resource_spent = np.dot(duration_array, choice_array)
    if(resource_spent != total_resource):
        return(0)
    else:
        return(1)

# np.array([start days]), np.array([duration]), np.array([choice], number_of_days_simulated)
def mod_calculate_opt(variable_params, total_resource, days):
    R0 = 2.2 
    T_inf = 2.9
    T_trans = T_inf/R0
    T_treat = 15

    N = 1e5
    I0 = 3.0

    intervention_day = np.array(variable_params['intervention_day'])
    intervention_duration = np.array(variable_params['intervention_duration'])
    intervention_choice = np.array(variable_params['intervention_choice'])

    assert(len(intervention_day) == len(intervention_duration))
    assert(len(intervention_duration) == len(intervention_choice))
    if(not check(intervention_day, intervention_duration, intervention_choice, total_resource)):
        return(100)
    
    params = [T_trans, T_treat, N, intervention_day, intervention_duration, intervention_choice]

    # S, E, I, R_mild, R_severe, R_severe_home, R_fatal, C, D
    state_init_values = [(N - I0)/N, I0/N, 0]
    
    solver = SEIR(params, state_init_values)
    sol = solver.solve_ode(time_step=1, total_no_of_days=days)
    states_int_array = (sol.y*N).astype('int')
    
    
    S_coeficeint=0
    I_coeficeint=1
    R_coeficeint=0
 
    coeficeint=[S_coeficeint,I_coeficeint,R_coeficeint]
    
    for i in range(days):
        grad1 = np.dot(coeficeint,np.sum(states_int_array/N, axis=1))
        
    return(grad1)

def tpe_opt(num_int, total_resource=30, day0=30):
    if(num_int==1):
        start_val = [start for start in range(day0, day0+365, 10)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource+1, 5)]

        variable_params = {
            'intervention_day' : [hp.choice('intervention_day', start_val)],
            'intervention_duration' : [hp.choice('intervention_duration', duration_val)],
            'intervention_choice' : [hp.choice('intervention_choice', choice_val)],
        }
        
    if(num_int==2):
        start_val = [start for start in range(day0, day0+365, 10)]
        choice_val = [0.25, 0.5, 1]
        duration_val = [duration for duration in range(10, 4*total_resource+1, 5)]

        variable_params = {
            'intervention_day' : [hp.choice('id_0', start_val),hp.choice('id_1', start_val)],
            'intervention_duration' : [hp.choice('du_0', duration_val),hp.choice('du_1', duration_val)],
            'intervention_choice' : [hp.choice('ic_0', choice_val),hp.choice('ic_1', choice_val)],
        }

    partial_calculate_opt = partial(mod_calculate_opt, total_resource=total_resource, days=400) 
    
    searchspace = variable_params
    
    trials = Trials()
    best = fmin(partial_calculate_opt,
                space=searchspace,
                algo=tpe.suggest,
                max_evals=3000,
                trials=trials)
    
    return(best, trials)

In [29]:
tpe_opt(num_int=1)


100%|██████████| 3000/3000 [01:06<00:00, 45.02trial/s, best loss: 14.99759]


({'intervention_choice': 2, 'intervention_day': 0, 'intervention_duration': 4},
 <hyperopt.base.Trials at 0x7f97e4bda9b0>)

In [57]:
tpe_opt(num_int=2)


100%|██████████| 3000/3000 [01:47<00:00, 27.79trial/s, best loss: 14.997539999999999]


({'du_0': 2, 'du_1': 2, 'ic_0': 2, 'ic_1': 1, 'id_0': 0, 'id_1': 3},
 <hyperopt.base.Trials at 0x7f98619a8b00>)

## Heuristic Based Method

In [58]:
from models.optim.sir_cont import SIR as SIR_CONT
# np.array([start days]), np.array([duration]), np.array([choice], number_of_days_simulated)
def run_seir(days, int_vec):
    R0 = 2.2 
    T_inf = 2.9
    T_trans = T_inf/R0
    T_treat = 15

    N = 1e5
    I0 = 3.0
    
    params = [T_trans, T_treat, N, int_vec]

    # S, E, I, R_mild, R_severe, R_severe_home, R_fatal, C, D
    state_init_values = [(N - I0)/N, I0/N, 0]
    
    solver = SIR_CONT(params, state_init_values)
    sol = solver.solve_ode(time_step=1, total_no_of_days=days)
    states_int_array = (sol.y*N).astype('int')
    
    derivatives = np.ones((days,3))
    for t in range(days):
        derivatives[t] = solver.get_derivative(t=t, y=states_int_array[:,t])
    
    S_coeficeint=0
    I_coeficeint=1
    R_coeficeint=0
    
    coeficeint=np.array([S_coeficeint,I_coeficeint,R_coeficeint])
    
    grad1 = np.ones(days)
    grad2 = np.ones(days)

    for i in range(days):
        grad2[i] = np.dot(coeficeint,(derivatives[i]/N))
        grad1[i] = np.dot(coeficeint,(states_int_array[:,i]/N))
    return(grad1)

In [59]:
total_resource = 30
resource_spent = 0
window = 5
int_vec = np.ones(400)
while(resource_spent<total_resource):
    curve = run_seir(days=400, int_vec=int_vec)
    arg_curve = (-curve).argsort()
    day_max = arg_curve[0]
    i=1
    while(int_vec[day_max]>=2):
        day_max = arg_curve[i]
        i += 1

    int_vec[day_max] += 0.25 
    resource_spent += 0.25

In [60]:
int_vec

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [61]:
curve = run_seir(days=400, int_vec=int_vec)
np.sum(curve)

14.9848

In [62]:
curve_base = run_seir(days = 400, int_vec=np.ones(400))
np.sum(curve_base)

14.99909

## Gradient Based Methods

In [63]:
import sys
sys.path.append('../..')
from models.optim.sir_cont import SIR as SIR_CONT
# np.array([start days]), np.array([duration]), np.array([choice], number_of_days_simulated)
def run_seir(days, int_vec):
    R0 = 2.2 
    T_inf = 2.9
    T_trans = T_inf/R0
    T_treat = 15

    N = 1e5
    I0 = 3.0
    
    params = [T_trans, T_treat, N, int_vec]

    # S, E, I, R_mild, R_severe, R_severe_home, R_fatal, C, D
    state_init_values = [(N - I0)/N, I0/N, 0]
    
    solver = SIR_CONT(params, state_init_values)
    sol = solver.solve_ode(time_step=1, total_no_of_days=days)
    states_int_array = (sol.y*N).astype('int')
    
    derivatives = np.ones((days,3))
    for t in range(days):
        derivatives[t] = solver.get_derivative(t=t, y=states_int_array[:,t])
    
    S_coeficeint=0
    I_coeficeint=1
    R_coeficeint=0
    
    coeficeint=np.array([S_coeficeint,I_coeficeint,R_coeficeint])
    
    grad1 = np.ones(days)
    grad2 = np.ones(days)

    for i in range(days):
        grad2[i] = np.dot(coeficeint,(derivatives[i]/N))
        grad1[i] = np.dot(coeficeint,(states_int_array[:,i]/N))
    return(grad1)

### Getting gradients wrt int_vec

In [64]:
def get_grad(int_vec):
    days = len(int_vec)
    num_pts = 20  # number of points used for derievative.
    grad_vec = np.ones(days)
    for i in range(days):
    #     i represents the ith dimension of T_transi
        val = int_vec[i]
        window_size = 0.005
        values = np.random.uniform(val-window_size, val+window_size, num_pts)
        f_values = np.ones_like(values)
        for j,value in enumerate(values):
            int_vec[i] = value
            f_values[j] = np.sum(run_seir(days, int_vec))
        values = np.append(values,val)
        int_vec[i] = val
        fval = np.sum(run_seir(days, int_vec))
        f_values = np.append(f_values,fval)
        derivative = np.gradient(f_values, values, edge_order=2)
        grad_vec[i] = derivative[-1]
    return(grad_vec)

### Formulating optimization problem

In [65]:
from pulp import *
def opt_step(grad):
    prob = LpProblem("Minimization over constrained space",LpMinimize)

    day_list = list((np.arange(400)).astype(str))

    int_days = LpVariable.dicts("day",day_list,lowBound=1,upBound=2,cat='Continuous')

#     grad = get_grad(np.ones(400))

    prob += lpSum([grad[int(i)]*int_days[i] for i in day_list]) #objective function
    prob += lpSum([int_days[f] for f in day_list]) <= 430.0 #TotalResource
    prob.solve()
    opt_val = np.ones(400)
    for v in prob.variables():
        idx = v.name.split('_')[-1]
        opt_val[int(idx)] = v.varValue
        
    return(opt_val)

### Adding constraints

In [5]:
def l2_diff(int_days, day_list):
    ret_val = []
    for day in day_list:
        if(day!='399'):
            val = int_days[str(int(day)+1)] - int_days[day]
            ret_val.append(val)
    return(ret_val)

In [6]:
# prob += lpSum([int_days[f] for f in day_list]) <= 430.0 #"TotalResource"
# diff_arr = l2_diff(int_days,day_list)
# prob += lpSum(diff_arr) <= 5 #"NumInterventions"

### Frank-Wolfe algo

In [66]:
num_iter = 5
int_vec = np.ones(400)
for i in range(num_iter):
    gradient = get_grad(int_vec)
    opt = opt_step(gradient)
    gamma = 2.0/(3+i)
    int_vec = int_vec + gamma*(opt-int_vec)
    print(int_vec)

/home/users/bhardwaj/env_covid/lib/python3.7/site-packages/pulp/pulp.py:1114: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


[1.         1.         1.         1.66666667 1.         1.
 1.66666667 1.66666667 1.66666667 1.         1.         1.66666667
 1.         1.         1.         1.         1.66666667 1.
 1.66666667 1.         1.         1.         1.66666667 1.
 1.         1.66666667 1.         1.66666667 1.66666667 1.
 1.         1.66666667 1.         1.66666667 1.         1.
 1.         1.         1.66666667 1.         1.66666667 1.
 1.         1.         1.66666667 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1. 

In [67]:
np.sum(run_seir(int_vec=int_vec,days=400))

14.99654

In [68]:
np.sum(run_seir(int_vec=np.ones(400),days=400))

14.99909

## RL Based approach

In [6]:
from models.optim.sir_discrete import SIR_Discrete
from sklearn.ensemble import ExtraTreesRegressor
import pickle
import time
from tqdm import tqdm, trange
import math

In [5]:
def is_fitted(sklearn_regressor):
    """Helper function to determine if a regression model from scikit-learn has
    ever been `fit`"""
    return hasattr(sklearn_regressor, 'n_outputs_')

In [7]:
class FittedQIteration(object):
    def __init__(self, regressor=None):
        """Initialize simulator and regressor. Can optionally pass a custom
        `regressor` model (which must implement `fit` and `predict` -- you can
        use this to try different models like linear regression or NNs)"""
        self.simulator = SIR_Discrete()
        self.regressor = regressor or ExtraTreesRegressor()

    def Q(self, states):
        """Return the Q function estimate of `states` for each action"""
        if not is_fitted(self.regressor):
            # If not fitted, return 0s in the right shape
            return np.zeros((len(states), self.simulator.num_actions))
        else:
            # Otherwise use the trained regression model
            return np.array([self.regressor.predict(self.encode(states, action))
                             for action in range(self.simulator.num_actions)]).T

    def policy(self, state, eps=0.1):
        """Return the epsilon-greedy action based on the current policy (or a
        random action if the Q function hasn't yet been estimated."""
        if np.random.rand() < eps or not is_fitted(self.regressor):
            return np.random.choice(self.simulator.num_actions)
        else:
            return self.Q([state]).argmax()

    def run_episode(self, eps=0.1, episode_length=200):
        """Run a single episode on the HIV simulator using the current policy.
        Can pass a custom `eps` to test out varying levels of randomness.
        Return the states visited, actions taken, and rewards received."""
        S = np.zeros((episode_length+1, self.simulator.num_states))
        A = np.zeros(episode_length)
        R = np.zeros(episode_length)

        self.simulator.reset()
        S[0] = self.simulator.STATE

        for t in range(episode_length):
            A[t] = self.policy(S[t], eps=eps)
            R[t], S[t+1] = self.simulator.perform(A[t])
        return S, A, R

    def fit_Q(self, episodes, num_iters=100, discount=0.98,episode_length=200):
        """Fit and re-fit the Q function using historical data for the
        specified number of `iters` at the specified `discount` factor"""
        S1 = np.vstack([ep[0][:-1] for ep in episodes])
        S2 = np.vstack([ep[0][1:] for ep in episodes])
        A = np.hstack([ep[1] for ep in episodes])
        R = np.hstack([ep[2] for ep in episodes])
        inputs = self.encode(S1, A)
        progress = tqdm(range(num_iters), file=sys.stdout,desc='num_iters')
        for _ in range(num_iters):
            progress.update(1)
            targets = R + discount * self.Q(S2).max(axis=1)
            self.regressor.fit(inputs, targets)
        progress.close()
    def fit(self, num_refits=10, num_episodes=15, episode_length=200, save=False):
        """Perform fitted-Q iteration. For `outer_iters` steps, gain
        `num_episodes` episodes worth of experience using the current policy
        (which is initially random), then fit or re-fit the Q-function. Return
        the full set of episode data."""
        episodes = []
        progress = tqdm(range(num_refits), file=sys.stdout,desc='num_refits')
        for i in range(num_refits):
            progress.update(1)
            for _ in range(num_episodes):
                episodes.append(self.run_episode(episode_length=episode_length))
            self.fit_Q(episodes=episodes, episode_length=episode_length)
            if save:
                with open('./fqi-regressor-iter-{}.pkl'.format(i+1), 'wb') as f:
                    pickle.dump(self.regressor, f)
        progress.close()
        return episodes

    def encode(self, states, actions):
        """Encode states and actions as a single input array, suitable for
        passing into the `fit` method of a scikit-learn model."""
        if isinstance(actions, int):
            actions = [actions] * len(states)
        actions = np.array([[a] for a in actions])
        states = np.array(states)
        return np.hstack([states, actions])

In [19]:
def run_rl():
    num_refits=10
    num_episodes=15
    episode_length=400
    env=FittedQIteration()
    episodes=env.fit( num_refits=num_refits, num_episodes=num_episodes,episode_length=episode_length)

    discount=0.98
    real_episodes=env.run_episode(eps=0,episode_length=episode_length)
    S=real_episodes[0]
    A=real_episodes[1]
    R=real_episodes[2]
    best_r=0
    for i in range(len(R)):
        best_r+=R[i]*(discount**i)


    random_action_episodes=env.run_episode(eps=1,episode_length=episode_length)
    S_r=random_action_episodes[0]
    A_r=random_action_episodes[1]
    R_r=random_action_episodes[2]
    random_r=0
    for i in range(len(R_r)):
        random_r+=R_r[i]*(discount**i)
    test_r=0    

    state=real_episodes[0][0]

    estimate_r=env.Q([state])[0]
    print("Estimate Reward(Discounted):")
    print(max(estimate_r))
    print("Real Reward(Discounted):")
    print(best_r)
    print("Random Action Reward(Discounted):")
    print(random_r)
    S_=[]
    R_=[]
    I_=[]
    for i in range(len(S)):
        S_.append(S[i][0])
        I_.append(S[i][1])
        R_.append(S[i][2])
    S_coeficeint=0
    I_coeficeint=1
    R_coeficeint=0
    
    coeficeint=np.array([S_coeficeint,I_coeficeint,R_coeficeint])
    states_int_array=np.array([S_,I_,R_])
    
    grad1 = np.ones(episode_length)

    for i in range(episode_length):
        grad1[i] = np.dot(coeficeint,states_int_array[:,i])
    return(grad1)    

In [20]:
np.sum(run_rl())


num_refits:   0%|          | 0/10 [00:00<?, ?it/s]

num_iters:   0%|          | 0/100 [00:00<?, ?it/s]

num_iters:   2%|▏         | 2/100 [00:00<00:15,  6.34it/s]

num_iters:   3%|▎         | 3/100 [00:00<00:26,  3.66it/s]

num_iters:   4%|▍         | 4/100 [00:01<00:35,  2.67it/s]

num_iters:   5%|▌         | 5/100 [00:02<00:45,  2.07it/s]

num_iters:   6%|▌         | 6/100 [00:02<00:50,  1.85it/s]

num_iters:   7%|▋         | 7/100 [00:03<00:53,  1.73it/s]

num_iters:   8%|▊         | 8/100 [00:04<00:59,  1.56it/s]

num_iters:   9%|▉         | 9/100 [00:05<00:59,  1.53it/s]

num_iters:  10%|█         | 10/100 [00:05<01:00,  1.50it/s]

num_iters:  11%|█         | 11/100 [00:06<01:00,  1.48it/s]

num_iters:  12%|█▏        | 12/100 [00:07<01:00,  1.45it/s]

num_iters:  13%|█▎        | 13/100 [00:07<01:01,  1.41it/s]

num_iters:  14%|█▍        | 14/100 [00:08<01:00,  1.42it/s]

num_iters:  15%|█▌        | 15/100 [00:09<01:00,  1.41it/s]

num_iters:  16%|█▌        | 16/100 [00:10<00:59, 

14.997822345481504